In [0]:
# Set for future print function
from __future__ import print_function

# Import PyTorch Libraries
import torch # Base library
import torch.nn as nn # To Define the neural network architecture as object
import torch.nn.functional as F # To define functional operation on NN object
import torch.optim as optim # Optimization function libraries SGD, ADAM
from torchvision import datasets, transforms #Utilizing existing dataset & transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #original network
        #self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 32, kernel_size = 3, padding = 1) #input 1x28x28, |32x1x3x3|, output 32x28x28, RF=3
        #self.conv2 = nn.Conv2d(in_channels = 32, out_channels = 64, kernel_size = 3, padding = 1) #input 32x28x28, |64x32x3x3|, output 64x28x28, RF=5

        #self.pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2) #64x28x28|2x2|64x14x14, RF=5, J=2
        #self.conv3 = nn.Conv2d(in_channels = 64, out_channels = 128, kernel_size = 3, padding = 1) #input 64x14x14, |128x64x3x3|, output 128x14x14, RF=10
        #self.conv4 = nn.Conv2d(in_channels = 128, out_channels = 256, kernel_size = 3, padding = 1) #input 128x14x14, |256x128x3x3|, output 256x14x14, RF=12

        #self.pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2) #input 256x14x14, |2x2|, output 256x7x7, RF=12
        #self.conv5 = nn.Conv2d(in_channels = 256, out_channels = 512, kernel_size = 3) #input 256x7x7, |512x256x3x3|, output 512x5x5, RF=24
        #self.conv6 = nn.Conv2d(in_channels = 512, out_channels = 1024, kernel_size = 3) #input 512x5x5, |1024x512x3x3|, output 1024x3x3, RF=26
        #self.conv7 = nn.Conv2d(in_channels = 1024, out_channels = 10, kernel_size = 3) #input 1024x3x3, |10x1024x3x3|, output 10x1x1, RF=28

        #modified
        self.conv1 = nn.Conv2d(1, 16, 3, padding = 1) #input 28x28x1, output 28x28x16, RF 3
        self.bn1 = nn.BatchNorm2d(16)
        self.do1 = nn.Dropout(0.15)

        self.conv2 = nn.Conv2d(16, 32, 3, padding = 1) #input 28x28x16, output 28x28x32, RF 3
        self.bn2 = nn.BatchNorm2d(32)
        self.do2 = nn.Dropout(0.15)

        self.pool1 = nn.MaxPool2d(2, 2) # input 28x28*32 output 14*14*32 RF 6

        self.conv3 = nn.Conv2d(32, 16, 3, padding=1) #input 14x14x32, output 14x14x16, RF 6
        self.bn3 = nn.BatchNorm2d(16)
        self.do3 = nn.Dropout(0.1)

        self.conv4 = nn.Conv2d(16, 16, 3, padding=1) #input 14*14*16 output 14*14*16 RF 6
        self.bn4 = nn.BatchNorm2d(16)
        self.do4 = nn.Dropout(0.1)

        self.pool2 = nn.MaxPool2d(2, 2) # input 14*14*16 op 7*7*16 RF 16

        self.conv5 = nn.Conv2d(16, 16, 3) # input 7*7*16 op 7*7*16 RF 24
        self.bn5 = nn.BatchNorm2d(16)
        self.do5 = nn.Dropout(0.1)

        self.conv6 = nn.Conv2d(16, 16, 3) # input 5*5*16 op 5*5*16 RF 32
        self.bn6 = nn.BatchNorm2d(16)
        self.do6 = nn.Dropout(0.05)

        self.conv7 = nn.Conv2d(16, 10, 3) # input 3*3*16 op 1*1*10 RF 40

    def forward(self, x):
        #original network
        #x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        #x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        #x = F.relu(self.conv6(F.relu(self.conv5(x))))
        #x = (self.conv7(x))
        #x = x.view(-1, 10)
        #return F.log_softmax(x)

        #modified
        x = self.pool1(F.relu(self.bn2(self.do2(self.conv2(F.relu(self.bn1(self.do1(self.conv1(x)))))))))
        x = self.pool2(F.relu(self.bn4(self.do4(self.conv4(F.relu(self.bn3(self.do3(self.conv3(x)))))))))
        x = F.relu(self.bn6(self.do6(self.conv6(self.bn5(self.do5(self.conv5(x)))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [0]:
# Model summary PyTorch
!pip install torchsummary

from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")

# Model summary device here.
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
           Dropout-2           [-1, 16, 28, 28]               0
       BatchNorm2d-3           [-1, 16, 28, 28]              32
            Conv2d-4           [-1, 32, 28, 28]           4,640
           Dropout-5           [-1, 32, 28, 28]               0
       BatchNorm2d-6           [-1, 32, 28, 28]              64
         MaxPool2d-7           [-1, 32, 14, 14]               0
            Conv2d-8           [-1, 16, 14, 14]           4,624
           Dropout-9           [-1, 16, 14, 14]               0
      BatchNorm2d-10           [-1, 16, 14, 14]              32
           Conv2d-11           [-1, 16, 14, 14]           2,320
          Dropout-12           [-1, 16, 14, 14]               0
      BatchNorm2d-13           [-1, 16, 14, 14]              32
        MaxPool2d-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
# tqdm - to visualize training progress
from tqdm import tqdm
test_accuracies = []

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_accuracies.append(100. * correct / len(test_loader.dataset))

In [0]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 20):
    print ('\n ********* EPOCH {} *********'.format(epoch))
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


 ********* EPOCH 1 *********


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:63: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0916, Accuracy: 9750/10000 (97.50%)


 ********* EPOCH 2 *********


  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0200, Accuracy: 9937/10000 (99.37%)


 ********* EPOCH 18 *********


  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0186, Accuracy: 9941/10000 (99.41%)


 ********* EPOCH 19 *********


loss=0.009935309179127216 batch_id=468: 100%|██████████| 469/469 [00:16<00:00, 28.32it/s]



Test set: Average loss: 0.0198, Accuracy: 9935/10000 (99.35%)



In [0]:

# Displaying the test accuracy and the threshold to passed.
print(test_accuracies)
threshold = 19*[99.4]
print (threshold)


[97.5, 98.69, 98.97, 99.1, 98.88, 99.1, 99.0, 99.17, 99.33, 99.25, 99.22, 99.26, 99.34, 99.38, 99.31, 99.38, 99.37, 99.41, 99.35]
[99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4, 99.4]
